In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.
'C:\Users\Andy' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import numpy as np
import datetime
import time
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, balanced_accuracy_score
import pydotplus
from IPython.display import Image
%matplotlib inline
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.linear_model import LogisticRegression
from imblearn.metrics import classification_report_imbalanced
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.combine import SMOTEENN
from imblearn.ensemble import BalancedRandomForestClassifier

In [3]:
ticker = 'MSFT'
start_date = '2011-01-01'
end_date = '2021-05-31'

stock_df = yf.download(ticker, start=start_date, end=end_date)
stock_df['Daily Return'] = stock_df['Close'].pct_change().dropna()
stock_df['Adj Daily Return'] = stock_df['Close'].pct_change().dropna().shift()
#stock_df['Daily Rolling Volatility'] = stock_df['Daily Return'].rolling(window=10).std()
stock_df['Adj Daily Rolling Volatility'] = stock_df['Daily Return'].rolling(window=10).std().shift()
stock_df['Decision'] = np.where(stock_df['Adj Daily Return'] > 0, 'Entry' , 'Exit')
stock_df.dropna(inplace=True)
stock_df.drop(['Close', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
stock_df.tail(20)

[*********************100%***********************]  1 of 1 completed


,Open,Daily Return,Adj Daily Return,Adj Daily Rolling Volatility,Decision
Date,,,,,
2021-05-03,253.399994,-0.001269,-0.001307,0.012104,Exit
2021-05-04,250.970001,-0.016160,-0.001269,0.012014,Exit
2021-05-05,249.059998,-0.005327,-0.016160,0.012742,Exit
2021-05-06,246.449997,0.013227,-0.005327,0.011890,Exit
2021-05-07,252.149994,0.010932,0.013227,0.012891,Entry
2021-05-10,250.869995,-0.020914,0.010932,0.012236,Entry
2021-05-11,244.550003,-0.003843,-0.020914,0.013265,Exit
2021-05-12,242.169998,-0.029363,-0.003843,0.013048,Exit
2021-05-13,241.800003,0.016862,-0.029363,0.013254,Exit


In [4]:
X = stock_df.copy()
X.drop(['Decision', 'Daily Return', 'Adj Daily Return'], axis=1, inplace=True)
X.dropna(inplace=True)
X.tail(20)

,Open,Adj Daily Rolling Volatility
Date,,
2021-05-03,253.399994,0.012104
2021-05-04,250.970001,0.012014
2021-05-05,249.059998,0.012742
2021-05-06,246.449997,0.011890
2021-05-07,252.149994,0.012891
2021-05-10,250.869995,0.012236
2021-05-11,244.550003,0.013265
2021-05-12,242.169998,0.013048
2021-05-13,241.800003,0.013254


In [5]:
y = stock_df['Decision'].values.reshape(-1,1)
y[:20]

array([['Entry'],
       ['Exit'],
       ['Exit'],
       ['Exit'],
       ['Entry'],
       ['Entry'],
       ['Entry'],
       ['Entry'],
       ['Exit'],
       ['Exit'],
       ['Entry'],
       ['Exit'],
       ['Exit'],
       ['Entry'],
       ['Entry'],
       ['Entry'],
       ['Exit'],
       ['Exit'],
       ['Exit'],
       ['Exit']], dtype=object)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
model = tree.DecisionTreeClassifier()
model = model.fit(X_train_scaled, y_train)

In [8]:
predictions = model.predict(X_test_scaled)

In [9]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    model = GradientBoostingClassifier(
        n_estimators=1000,
        learning_rate=learning_rate,
        max_features=2,
        max_depth=3,
        random_state=0)
    model.fit(X_train_scaled,y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.5f}".format(
        model.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.5f}".format(
        model.score(
            X_test_scaled,
            y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.88497
Accuracy score (validation): 0.52761

Learning rate:  0.1
Accuracy score (training): 0.95706
Accuracy score (validation): 0.50920

Learning rate:  0.25
Accuracy score (training): 0.99898
Accuracy score (validation): 0.51687

Learning rate:  0.5
Accuracy score (training): 1.00000
Accuracy score (validation): 0.50767

Learning rate:  0.75
Accuracy score (training): 1.00000
Accuracy score (validation): 0.50920

Learning rate:  1
Accuracy score (training): 1.00000
Accuracy score (validation): 0.52761



In [10]:
predictions_df = pd.DataFrame({"Actual": y_test.ravel(),"Prediction": predictions}, index=stock_df.index[-len(y_test):])
predictions_df.head(20)

,Actual,Prediction
Date,,
2018-10-25,Entry,Entry
2018-10-26,Entry,Exit
2018-10-29,Entry,Entry
2018-10-30,Entry,Exit
2018-10-31,Exit,Exit
2018-11-01,Entry,Entry
2018-11-02,Entry,Entry
2018-11-05,Exit,Exit
2018-11-06,Exit,Exit


In [11]:
model = GradientBoostingClassifier(
    n_estimators=1000,
    learning_rate=1,
    max_features=2,
    max_depth=3,
    random_state=0)

# Fit the model
model.fit(X_train_scaled,y_train.ravel())

# Score the model
print("Accuracy score (training): {0:.5f}".format(
    model.score(
        X_train_scaled,
        y_train)))
print("Accuracy score (validation): {0:.5f}".format(
    model.score(
        X_test_scaled,
        y_test)))

Accuracy score (training): 1.00000
Accuracy score (validation): 0.52761


In [12]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

acc_score = accuracy_score(y_test, predictions)

In [13]:
print('Confusion Matrix')
display(cm_df)
print()
print()
print(f'Accuracy Score: {acc_score:.5f}')
print()
print()
print('Classification Report')
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,180,151
Actual Exit,169,152




Accuracy Score: 0.50920


Classification Report
              precision    recall  f1-score   support

       Entry       0.52      0.54      0.53       331
        Exit       0.50      0.47      0.49       321

    accuracy                           0.51       652
   macro avg       0.51      0.51      0.51       652
weighted avg       0.51      0.51      0.51       652



In [14]:
dot_data = tree.export_graphviz(
    model.estimators_[9, 0],
    out_file=None, filled=True,
    rounded=True,
    special_characters=True,
    proportion=True,
)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())


InvocationException: GraphViz's executables not found

## Random Oversampling

In [15]:
ros = RandomOverSampler(random_state=78)
X_over_resampled, y_over_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_over_resampled)

Counter({'Exit': 1032, 'Entry': 1032})

In [16]:
model_over_resample = LogisticRegression(solver='lbfgs', random_state=78)
model_over_resample.fit(X_over_resampled, y_over_resampled)
y_pred_over = model.predict(X_test_scaled)
cm_over = confusion_matrix(y_test, y_pred_over)
cm_over_df = pd.DataFrame(
    cm_over, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)
bac_over = balanced_accuracy_score(y_test, y_pred_over)

In [17]:
print('Confusion Matrix')
display(cm_over_df)
print()
print()
print(f'Balance Accuracy Score: {bac_over:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_over))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,196,135
Actual Exit,173,148




Balance Accuracy Score: 0.52660


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.53      0.59      0.46      0.56      0.52      0.28       331
       Exit       0.52      0.46      0.59      0.49      0.52      0.27       321

avg / total       0.53      0.53      0.53      0.53      0.52      0.27       652



## SMOTE

In [18]:
X_resampled_smote, y_resampled_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)

Counter(y_resampled_smote)

Counter({'Exit': 1032, 'Entry': 1032})

In [19]:
model_smote = LogisticRegression(solver='lbfgs', random_state=78)
model_smote.fit(X_resampled_smote, y_resampled_smote)
y_pred_smote = model_smote.predict(X_test_scaled)
bac_smote = balanced_accuracy_score(y_test, y_pred_smote)
cm_smote = confusion_matrix(y_test, y_pred_smote)
cm_smote_df = pd.DataFrame(
    cm_smote, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

In [20]:
print('Confusion Matrix')
display(cm_smote_df)
print()
print()
print(f'Balance Accuracy Score: {bac_smote:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_smote))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,149,182
Actual Exit,111,210




Balance Accuracy Score: 0.55218


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.57      0.45      0.65      0.50      0.54      0.29       331
       Exit       0.54      0.65      0.45      0.59      0.54      0.30       321

avg / total       0.55      0.55      0.55      0.55      0.54      0.29       652



## Undersampling

In [21]:
rus = RandomUnderSampler(random_state=78)
X_under_resampled, y_under_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_under_resampled)

Counter({'Exit': 1032, 'Entry': 1032})

In [22]:
model_under_resample = LogisticRegression(solver='lbfgs', random_state=78)
model_under_resample.fit(X_under_resampled, y_under_resampled)
y_pred_under = model_under_resample.predict(X_test_scaled)
bac_under = balanced_accuracy_score(y_test, y_pred_under)
cm_under = confusion_matrix(y_test, y_pred_under)
cm_under_df = pd.DataFrame(
    cm_under, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

In [23]:
print('Confusion Matrix')
display(cm_under_df)
print()
print()
print(f'Balance Accuracy Score: {bac_under:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_under))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,160,171
Actual Exit,120,201




Balance Accuracy Score: 0.55478


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.57      0.48      0.63      0.52      0.55      0.30       331
       Exit       0.54      0.63      0.48      0.58      0.55      0.31       321

avg / total       0.56      0.55      0.56      0.55      0.55      0.30       652



## Cluster Centroid Undersampling

In [24]:
cluster = ClusterCentroids(random_state=78)
X_resampled_cc, y_resampled_cc = cluster.fit_resample(X_train_scaled, y_train)
Counter(y_resampled_cc)

Counter({'Entry': 924, 'Exit': 924})

In [25]:
model_resample_cc = LogisticRegression(solver='lbfgs', random_state=78)
model_resample_cc.fit(X_resampled_cc, y_resampled_cc)
y_pred_cc = model_resample_cc.predict(X_test_scaled)
bac_cc = balanced_accuracy_score(y_test, y_pred_cc)
cm_cc = confusion_matrix(y_test, y_pred_cc)
cm_cc_df = pd.DataFrame(
    cm_cc, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

In [26]:
print('Confusion Matrix')
display(cm_cc_df)
print()
print()
print(f'Balance Accuracy Score: {bac_cc:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_cc))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,129,202
Actual Exit,97,224




Balance Accuracy Score: 0.54377


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.57      0.39      0.70      0.46      0.52      0.26       331
       Exit       0.53      0.70      0.39      0.60      0.52      0.28       321

avg / total       0.55      0.54      0.55      0.53      0.52      0.27       652



## SMOTEEN

In [27]:
sm = SMOTEENN(random_state=78)
X_resampled_smoteenn, y_resampled_smoteenn = sm.fit_resample(X_train_scaled, y_train)
Counter(y_resampled_smoteenn)

Counter({'Entry': 161, 'Exit': 175})

In [28]:
model_smoteenn = LogisticRegression(solver='lbfgs', random_state=78)
model_smoteenn.fit(X_resampled_smoteenn, y_resampled_smoteenn)
y_pred_smoteenn = model_smoteenn.predict(X_test_scaled)
bac_smoteenn = balanced_accuracy_score(y_test, y_pred_smoteenn)
cm_smoteenn = confusion_matrix(y_test, y_pred_smoteenn)
cm_smoteenn_df = pd.DataFrame(
    cm_smoteenn, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

In [29]:
print('Confusion Matrix')
display(cm_smoteenn_df)
print()
print()
print(f'Balance Accuracy Score: {bac_smoteenn:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_smoteenn))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,132,199
Actual Exit,96,225




Balance Accuracy Score: 0.54986


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.58      0.40      0.70      0.47      0.53      0.27       331
       Exit       0.53      0.70      0.40      0.60      0.53      0.29       321

avg / total       0.56      0.55      0.55      0.54      0.53      0.28       652



## Balanced Random Forest Classifier

In [30]:
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=78)
brf.fit(X_train_scaled, y_train)
y_pred_brf = brf.predict(X_test_scaled)
bac_brf = balanced_accuracy_score(y_test, y_pred_brf)
cm_brf = confusion_matrix(y_test, y_pred_brf)
cm_brf_df = pd.DataFrame(
    cm_brf, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

C:\Users\Andy L\anaconda3\envs\alpacaenv\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [31]:
print('Confusion Matrix')
display(cm_brf_df)
print()
print()
print(f'Balance Accuracy Score: {bac_brf:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_brf))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,174,157
Actual Exit,159,162




Balance Accuracy Score: 0.51518


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.52      0.53      0.50      0.52      0.52      0.27       331
       Exit       0.51      0.50      0.53      0.51      0.52      0.26       321

avg / total       0.52      0.52      0.52      0.52      0.52      0.27       652

